In [43]:
import numpy as np
import tensorflow as tf
import argparse
import glob
import os
import csv

imagePath = '/home/waleed/flower_photos/f.jpg'
modelFullPath = '/home/waleed/flower_photos/output_graph.pb'
#labelsFullPath = '/tmp/output_labels.txt'
indexpath = '/home/waleed/flower_photos/index.csv'

def create_graph():
    """Creates a graph from saved GraphDef file and returns a saver."""
    # Creates graph from saved graph_def.pb.
    with tf.gfile.FastGFile(modelFullPath, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        _ = tf.import_graph_def(graph_def, name='')

def run_on_image():
    answer = None

    if not tf.gfile.Exists(imagePath):
        tf.logging.fatal('File does not exist %s', imagePath)
        return answer

    image_data = tf.gfile.FastGFile(imagePath, 'rb').read()

    # Creates graph from saved GraphDef.
    create_graph()

    with tf.Session() as sess:

        next_to_last_tensor = sess.graph.get_tensor_by_name('pool_3:0')
        values = sess.run(next_to_last_tensor,{'DecodeJpeg/contents:0': image_data})
        values = np.squeeze(values)
        

    return values
if __name__ == '__main__':
    #run_inference_on_image()
    features=run_on_image()
    print features
    # saving the features
    output = open(indexpath, "w")
    features = [str(f) for f in features]
    output.write("%s,%s\n" % ("imageID", ",".join(features)))
    output.close()

[ 0.1099373   0.23045646  0.14808054 ...,  0.18244718  0.00371732
  0.14343837]


In [41]:
def chi2_distance(self, histA, histB, eps = 1e-10):
    # compute the chi-squared distance
    d = 0.5 * np.sum([((a - b) ** 2) / (a + b + eps)
        for (a, b) in zip(histA, histB)])

    # return the chi-squared distance
    return d

In [42]:
def search(queryFeatures, limit = 10):
    # initialize our dictionary of results
    results = {}

    # open the index file for reading
    with open(indexpath) as f:
        # initialize the CSV reader
        reader = csv.reader(f)

        # loop over the rows in the index
        for row in reader:
            # parse out the image ID and features, then compute the
            # chi-squared distance between the features in our index
            # and our query features
            features = [float(x) for x in row[1:]]
            d = self.chi2_distance(features, queryFeatures)

            # now that we have the distance between the two feature
            # vectors, we can udpate the results dictionary -- the
            # key is the current image ID in the index and the
            # value is the distance we just computed, representing
            # how 'similar' the image in the index is to our query
            results[row[0]] = d

        # close the reader
        f.close()

    # sort our results, so that the smaller distances (i.e. the
    # more relevant images are at the front of the list)
    results = sorted([(v, k) for (k, v) in results.items()])

    # return our (limited) results
    return results[:limit]
